# Sakila EDA - DuckDB + Pandas

In [ ]:
import duckdb

# Opening a connection to duckdb database
# So I don't have to open/close/read it every query
conn = duckdb.connect("../data/sakila.duckdb")

# Making schema "staging" the default searchpath
# so i don't have to write it out every query
conn.sql("SET search_path='staging';")

### a) Which movies are longer than three hours? (Showing title and length)

In [ ]:
conn.sql("""--sql
SELECT
    title,
    length
FROM
    film
WHERE
    length > 180
ORDER BY
    length DESC;
""").df().head(10)

,title,length
0,WORST BANGER,185
1,CHICAGO NORTH,185
2,CONTROL ANTHEM,185
3,DARN FORRESTER,185
4,SWEET BROTHERHOOD,185
5,GANGS PRIDE,185
6,HOME PITY,185
7,SOLDIERS EVOLUTION,185
8,POND SEATTLE,185
9,MUSCLE BRIGHT,185


### b) Which movies have the word "love" in its title? (Showing title, rating, length, description)

In [ ]:
conn.sql("""--sql
SELECT
	title,
    rating,
    length,
    description
FROM
    film
WHERE
    title ILIKE '%love%';
""").df()

,title,rating,length,description
0,GRAFFITI LOVE,PG,117,A Unbelieveable Epistle of a Sumo Wrestler And...
1,IDAHO LOVE,PG-13,172,A Fast-Paced Drama of a Student And a Crocodil...
2,IDENTITY LOVER,PG-13,119,A Boring Tale of a Composer And a Mad Cow who ...
3,INDIAN LOVE,NC-17,135,A Insightful Saga of a Mad Scientist And a Mad...
4,LAWRENCE LOVE,NC-17,175,A Fanciful Yarn of a Database Administrator An...
5,LOVE SUICIDES,R,181,A Brilliant Panorama of a Hunter And a Explore...
6,LOVELY JINGLE,PG,65,A Fanciful Yarn of a Crocodile And a Forensic ...
7,LOVER TRUMAN,G,75,A Emotional Yarn of a Robot And a Boy who must...
8,LOVERBOY ATTACKS,PG-13,162,A Boring Story of a Car And a Butler who must ...
9,STRANGELOVE DESIRE,NC-17,103,A Awe-Inspiring Panorama of a Lumberjack And a...


### c) Longest, shortest, median and average length movies

In [ ]:
# Adding 'minutes' to make it more readable

conn.sql("""--sql
SELECT
    MIN(length) || ' ' || 'minutes' as shortest_movie,
    MAX(length) || ' ' || 'minutes'  as longest_movie,
    AVG(length)::INT || ' ' || 'minutes'  as average_movie_length,
    MEDIAN(length)::INT  || ' ' || 'minutes' as median_movie_length
FROM
    film
""").df()

,shortest_movie,longest_movie,average_movie_length,median_movie_length
0,46 minutes,185 minutes,115 minutes,114 minutes


### d) 10 most expensive movie to rent per day

In [ ]:
# Rental duration is for how long you can have the movie
# So by dividing rate with duration, I get rent per day

rent_film = conn.sql("""--sql
SELECT
    title,
    release_year,
    description,
    (rental_rate / rental_duration) AS rent_per_day
FROM
    film
ORDER BY
    rent_per_day DESC;
""").df()

rent_film.head(10)

,title,release_year,description,rent_per_day
0,PATHS CONTROL,2006,A Astounding Documentary of a Butler And a Cat...,1.663333
1,BACKLASH UNDEFEATED,2006,A Stunning Character Study of a Mad Scientist ...,1.663333
2,BEHAVIOR RUNAWAY,2006,A Unbelieveable Drama of a Student And a Husba...,1.663333
3,VIRTUAL SPOILERS,2006,A Fateful Tale of a Database Administrator And...,1.663333
4,TEEN APOLLO,2006,A Awe-Inspiring Drama of a Dog And a Man who m...,1.663333
5,TELEGRAPH VOYAGE,2006,A Fateful Yarn of a Husband And a Dog who must...,1.663333
6,BILKO ANONYMOUS,2006,A Emotional Reflection of a Teacher And a Man ...,1.663333
7,HEARTBREAKERS BRIGHT,2006,A Awe-Inspiring Documentary of a A Shark And a...,1.663333
8,CLERKS ANGELS,2006,A Thrilling Display of a Sumo Wrestler And a G...,1.663333
9,TIES HUNGER,2006,A Insightful Saga of a Astronaut And a Explore...,1.663333


### e) Actor who played in most movies

In [ ]:
# Connecting first name and last name for readability
actors_in_movies = conn.sql("""--sql
SELECT
    a.first_name || ' ' || a.last_name AS actor_name,
    f.title,
FROM
    actor a
    LEFT JOIN film_actor fa ON fa.actor_id = a.actor_id
    LEFT JOIN film f ON f.film_id = fa.film_id;
""").df()

actors_in_movies["actor_name"].value_counts().head(5)

actor_name
SUSAN DAVIS       54
GINA DEGENERES    42
WALTER TORN       41
MARY KEITEL       40
MATTHEW CARREY    39
Name: count, dtype: int64

### f) Adding some interesting insights on my own (with a little help from my friend ChatGPT)

#### f.1) What movie categories are most rented?

In [ ]:
# Rental id is unique per rental
# so by counting rental_ids I get total rentals

conn.sql("""--sql
SELECT
    c.name AS category,
    COUNT(r.rental_id) AS total_rentals
FROM
    category c
    JOIN film_category fc ON fc.category_id = c.category_id
    JOIN film f ON f.film_id = fc.film_id
    JOIN inventory i ON f.film_id = i.film_id
    JOIN rental r ON i.inventory_id = r.inventory_id
GROUP BY
    c.name
ORDER BY
    total_rentals DESC
LIMIT
    10;
""").df()


,category,total_rentals
0,Sports,1179
1,Animation,1166
2,Action,1112
3,Sci-Fi,1101
4,Family,1096
5,Drama,1060
6,Documentary,1050
7,Foreign,1033
8,Games,969
9,Children,945


#### f.2) Which customers are most active overall and per month?

In [ ]:
#These costumers has rented the most movies, in total

conn.sql("""--sql
SELECT
    cu.first_name || ' ' || cu.last_name as customer,
    COUNT(r.rental_id) AS total_rentals
FROM
    customer cu
    JOIN rental r ON r.customer_id = cu.customer_id
GROUP BY
    cu.first_name,
    cu.last_name
ORDER BY
    total_rentals DESC
LIMIT
    10;
""").df()

,customer,total_rentals
0,ELEANOR HUNT,46
1,KARL SEAL,45
2,MARCIA DEAN,42
3,CLARA SHAW,42
4,TAMMY SANDERS,41
5,SUE PETERS,40
6,WESLEY BULL,40
7,MARION SNYDER,39
8,TIM CARY,39
9,RHONDA KENNEDY,39


In [ ]:
#Using date trunc to get month out of rental_date, and then grouping with
#customer to get largest montly_rentals from each customer each month.

per_month = conn.sql("""--sql
SELECT
    DATE_TRUNC ('month', rental_date) AS rental_month,
    cu.first_name || ' ' || cu.last_name AS customer,
    COUNT(*) AS monthly_rentals
FROM
    rental r
    JOIN customer cu ON cu.customer_id = r.customer_id
GROUP BY
    rental_month, customer
ORDER BY
    monthly_rentals DESC;
""").df()

per_month.head(10)

,rental_month,customer,monthly_rentals
0,2005-07-01,ELEANOR HUNT,22
1,2005-07-01,CRYSTAL FORD,21
2,2005-07-01,TAMMY SANDERS,20
3,2005-07-01,MARCIA DEAN,20
4,2005-07-01,KARL SEAL,19
5,2005-07-01,JUSTIN NGO,19
6,2005-07-01,TERRENCE GUNDERSON,19
7,2005-07-01,BRANDON HUEY,19
8,2005-07-01,RHONDA KENNEDY,19
9,2005-07-01,LOIS BUTLER,19


#### f.3) Which films generate the highest total revenue?

In [ ]:
#Using sum to add all amount-value to get a total

conn.sql("""--sql
SELECT
    f.title,
    SUM(p.amount) AS total_amount
FROM
    film f
    JOIN inventory i ON i.film_id = f.film_id
    JOIN rental r ON r.inventory_id = i.inventory_id
    JOIN payment p ON p.rental_id = r.rental_id
GROUP BY
    f.title
ORDER BY
    total_amount DESC
LIMIT
    10;
        """).df()

,title,total_amount
0,TELEGRAPH VOYAGE,231.73
1,WIFE TURN,223.69
2,ZORRO ARK,214.69
3,GOODFELLAS SALUTE,209.69
4,SATURDAY LAMBS,204.72
5,TITANS JERK,201.71
6,TORQUE BOUND,198.72
7,HARRY IDAHO,195.70
8,INNOCENT USUAL,191.74
9,HUSTLER PARTY,190.78


#### f.4) Which categories have the longest average film length?

In [ ]:
# Type-casting to int for readability

conn.sql("""--sql
SELECT
    c.name AS category,
    AVG(f.length)::INT AS average_movie_length
FROM
    category c
    JOIN film_category fc ON fc.category_id = c.category_id
    JOIN film f ON f.film_id = fc.film_id
GROUP BY
    category
ORDER BY
    average_movie_length DESC
LIMIT
    10;
    """).df()

,category,average_movie_length
0,Games,128
1,Sports,128
2,Foreign,122
3,Drama,121
4,Comedy,116
5,Family,115
6,Music,114
7,Travel,113
8,Action,112
9,Horror,112


#### f.5) Which store is no. 1 in rentals and revenue?

In [ ]:
conn.sql("""--sql
SELECT
    s.store_id,
    COUNT(r.rental_id) as total_rental,
    SUM(p.amount) as total_revenue
FROM
    store s
    JOIN customer cu ON cu.store_id = s.store_id
    JOIN rental r ON r.customer_id = cu.customer_id
    JOIN payment p ON p.rental_id = r.rental_id
GROUP BY
    s.store_id
ORDER BY
    total_revenue DESC;
""").df()

,store_id,total_rental,total_revenue
0,1,8747,36997.53
1,2,7297,30409.03


In [ ]:
conn.close()